In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os

from sqlalchemy import create_engine

In [3]:
# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')
fbi_code = pd.read_csv('input_data/FBI_Code.csv', encoding = "ISO-8859-1")
#fbi_code = pd.read_csv('input_data/FBI_Code.csv',  encoding= 'unicode_escape')
iucr_code = pd.read_csv('input_data/IUCR_Codes.csv', encoding = "ISO-8859-1")

In [4]:
# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))

679438


In [5]:
#Rafael

In [6]:
#Loading the FBI cvs file
df = fbi_code
df.head()

,Code,Description,Index,Violent/Property
0,01A,HOMICIDE  1ST OR 2ND DEGREE (INDEX),INDEX,VIOLENT INDEX
1,01B,INVOLUNTARY MANSLAUGHTER / RECKLESS HOMICIDE W...,NON-INDEX,NON-INDEX
2,02,CRIMINAL SEXUAL ASSAULT (INDEX),INDEX,VIOLENT INDEX
3,03,ROBBERY (INDEX),INDEX,VIOLENT INDEX
4,04,AGGRAVATED ASSAULT / BATTERY (INDEX),INDEX,VIOLENT INDEX


In [7]:
#Loading the IUCS cvs file
df2 = iucr_code
df2.head(10)

,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,INDEX CODE,ACTIVE
0,110,HOMICIDE,FIRST DEGREE MURDER,I,True
1,130,HOMICIDE,SECOND DEGREE MURDER,I,True
2,141,HOMICIDE,INVOLUNTARY MANSLAUGHTER,N,True
3,142,HOMICIDE,RECKLESS HOMICIDE,N,True
4,261,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - HANDGUN,I,True
5,262,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER FIREARM,I,True
6,263,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - KNIFE / CUTTING INSTRUMENT,I,True
7,264,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,I,True
8,265,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,I,True
9,266,CRIMINAL SEXUAL ASSAULT,PREDATORY,I,True


In [8]:
#dropping unnecessary columns
df = df.drop(df.columns[[2,3]], axis=1)
df.columns=['code', 'description']
df.head()


,code,description
0,01A,HOMICIDE  1ST OR 2ND DEGREE (INDEX)
1,01B,INVOLUNTARY MANSLAUGHTER / RECKLESS HOMICIDE W...
2,02,CRIMINAL SEXUAL ASSAULT (INDEX)
3,03,ROBBERY (INDEX)
4,04,AGGRAVATED ASSAULT / BATTERY (INDEX)


In [ ]:
df.columns

In [9]:
#dropping unnecessary columns
df2 = df2.drop(df2.columns[[3,4]], axis=1)
df2.head()



,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION
0,110,HOMICIDE,FIRST DEGREE MURDER
1,130,HOMICIDE,SECOND DEGREE MURDER
2,141,HOMICIDE,INVOLUNTARY MANSLAUGHTER
3,142,HOMICIDE,RECKLESS HOMICIDE
4,261,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - HANDGUN


In [10]:
#Checking and Removing duplicates rows
df = df.drop_duplicates()
df.head()

#writting csv file for import into pgadmin data base
#df.to_csv(r'./Output_data/FBI_code.csv', encoding='utf-8', index=False)

,code,description
0,01A,HOMICIDE  1ST OR 2ND DEGREE (INDEX)
1,01B,INVOLUNTARY MANSLAUGHTER / RECKLESS HOMICIDE W...
2,02,CRIMINAL SEXUAL ASSAULT (INDEX)
3,03,ROBBERY (INDEX)
4,04,AGGRAVATED ASSAULT / BATTERY (INDEX)


In [11]:
#Checking and Removing duplicates rows
df2 = df2.drop_duplicates()
df2.head()

,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION
0,110,HOMICIDE,FIRST DEGREE MURDER
1,130,HOMICIDE,SECOND DEGREE MURDER
2,141,HOMICIDE,INVOLUNTARY MANSLAUGHTER
3,142,HOMICIDE,RECKLESS HOMICIDE
4,261,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - HANDGUN


In [12]:
#Rename Columns
df2 = df2.rename(columns=({'PRIMARY DESCRIPTION':'PRIMARY_DESCRIPTION','SECONDARY DESCRIPTION':'SECONDARY_DESCRIPTION'}))
df2.head()

#writting csv file for import into pgadmin data base
#df2.to_csv(r'./Output_data/ICUS_code.csv', encoding='utf-8', index=False)

,IUCR,PRIMARY_DESCRIPTION,SECONDARY_DESCRIPTION
0,110,HOMICIDE,FIRST DEGREE MURDER
1,130,HOMICIDE,SECOND DEGREE MURDER
2,141,HOMICIDE,INVOLUNTARY MANSLAUGHTER
3,142,HOMICIDE,RECKLESS HOMICIDE
4,261,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - HANDGUN


In [13]:
#Artem
# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

#dropping unnecessary columns
crimes_data = crimes_data.drop(columns=['Latitude','Location','Longitude','X Coordinate','Y Coordinate','Updated On'])
# crimes_data.shape[0]

# removing duplicate values
crimes_data = crimes_data.drop_duplicates()
# crimes_data.shape[0]

# removing missing values
crimes_data = crimes_data.dropna()
# crimes_data.shape[0]

# Creating a function which will remove extra leading and tailing whitespace from the data.
def whitespace_remover(crimes_data):
   
    # iterating over the columns
    for i in crimes_data.columns:
         
        # checking datatype of each columns
        if crimes_data[i].dtype == 'object':
             
            # applying strip function on column
            crimes_data[i] = crimes_data[i].map(str.strip)
        else:
             
            # if condn. is False then it will do nothing.
            pass
        
# applyting whitespace_remover function on dataframe        
whitespace_remover(crimes_data)

#Rename Columns
crimes_data = crimes_data.rename(columns=({'Case Number':'Case_Number',
                                           'Location Description':'Location_Description',
                                           'Community Area':'Community_Area',
                                          'FBI Code':'FBI_Code'}))


#writting csv file for import into pgadmin data base
#crimes_data.to_csv(r'./Output_data/crimes_data.csv', encoding='utf-8', index=False)
crimes_data.head(20)

,ID,Case_Number,Date,Block,IUCR,Location_Description,Arrest,Domestic,Beat,District,District_name,District_population,Ward,Community_Area,FBI_Code
1,11554737,JC102447,1/1/2019 20:00,077XX S ABERDEEN ST,051A,STREET,False,True,612,6,Lake View,103050,17.0,71.0,04A
2,11552577,JC100040,1/1/2019 0:31,032XX W LAWRENCE AVE,1310,RESTAURANT,False,False,1713,17,Dunning,43147,33.0,14.0,14
3,11552587,JC100034,1/1/2019 0:05,006XX E 83RD PL,1310,RESIDENCE,False,False,632,6,Lake View,103050,6.0,44.0,14
4,11552596,JC100045,1/1/2019 0:03,001XX W HURON ST,430,HOTEL/MOTEL,False,False,1832,18,Montclare,14401,42.0,8.0,04B
5,11552605,JC100030,1/1/2019 0:01,004XX N MONTICELLO AVE,143A,ALLEY,True,False,1122,11,Jefferson Park,26216,27.0,23.0,15
6,11552609,JC100028,1/1/2019 0:05,013XX S CENTRAL PARK AVE,486,APARTMENT,False,True,1011,10,Norwood Park,38303,24.0,29.0,08B
7,11552610,JC100044,1/1/2019 0:02,031XX W 25TH ST,1310,RESIDENCE,False,False,1033,10,Norwood Park,38303,12.0,30.0,14
8,11552614,JC100052,1/1/2019 0:28,056XX W DIVERSEY AVE,470,BAR OR TAVERN,True,False,2514,25,Austin,96557,31.0,19.0,24
9,11552615,JC100069,1/1/2019 0:30,017XX E 74TH ST,1310,APARTMENT,False,False,324,3,Uptown,57182,8.0,43.0,14
10,11552616,JC100027,1/1/2019 0:03,017XX N MAYFIELD AVE,143A,RESIDENTIAL YARD (FRONT/BACK),True,False,2531,25,Austin,96557,29.0,25.0,15


In [ ]:
# create database connection
connection_string = "postgres:postgressql@localhost:5432/new_crime_db"
engine = create_engine(f'postgresql://{connection_string}')
# Confirm tables
engine.table_names()

In [ ]:
# rename columns to lower case 
df2.rename(columns={'IUCR':'iucr','PRIMARY_DESCRIPTION':'primary_description','SECONDARY_DESCRIPTION':'secondary_description' }, inplace=True)
crimes_data.columns=['id', 'case_number', 'date', 'block', 'iucr','location_description',
                     'arrest', 'domestic','beat','district', 'district_name', 'district_population',
                     'ward', 'community_area', 'fbi_code'
                    ]


In [ ]:
#load dataframe into database

df2.to_sql(name='iucr_codes', con=engine, if_exists='append', index=False)
df.to_sql(name='fbi_description', con=engine, if_exists='append', index=False)

In [ ]:
crimes_data.to_sql(name='crime_table', con=engine, if_exists='append', index=False)